In [1]:
#########################################
##The model deals with the prediction  ##
##predict required commodities for     ##
##each county each month and the total ##
## required for the next years         ##
#########################################

In [44]:
import pandas as pd

df  = pd.read_csv('data.csv')

#df.head()

In [42]:
#create a timestamp(the format is YYYY-MM-DD)
datetime  = []

def get_datetime():
    datetime.clear()
    for timestamp in df['periodid']:

        dates = str(timestamp)[:4]+"-"+str(timestamp)[-2:]

        datetime.append(dates)

get_datetime()
#print(datetime)

In [4]:
#per county and loop for all
#rename the columns
new_names  = {"Baringo County": 'baringo', "Bomet County": 'bomet', "Bungoma County": 'bungoma', "Busia County": 'busia', "Elgeyo Marakwet County": 'elgeyo',
               "Embu County": 'embu', "Garissa County": 'garissa', "Homa Bay County": 'homabay', "Isiolo County": 'isiolo', "Kajiado County": 'kajiado',
                "Kakamega County": 'kakamega', "Kericho County": 'kericho', "Kiambu County": 'kiambu', "Kilifi County": 'kilifi', "Kirinyaga County": 'kirinyaga',
                "Kisii County": 'kisii', "Kisumu County": 'kisumu', "Kitui County": 'kitui', "Kwale County": 'kwale', "Laikipia County": 'laikipia', "Lamu County": 'lamu',
                "Machakos County": 'machakos', "Makueni County": 'makueni', "Mandera County": 'mandera', "Marsabit County": 'marsabit', "Meru County": 'meru', "Migori County": 'migori',
                "Mombasa County": 'mombasa', "Muranga County": 'muranga', "Nairobi County": 'nairobi', "Nakuru County": 'nakuru', "Nandi County": 'nandi', "Narok County": 'narok',
                "Nyamira County": 'nyamira', "Nyandarua County": 'nyandarua', "Nyeri County": 'nyeri', "Samburu County": 'samburu', "Siaya County": 'siaya', "Taita Taveta County": 'taita', 
                "Tana River County": 'tanariver', "Tharaka Nithi County": 'tharaka', "Trans Nzoia County": 'transzoia', "Turkana County": 'turkana', "Uasin Gishu County": 'uasingishu',
                "Vihiga County": 'vihiga', "Wajir County": 'wajir', "West Pokot County": 'westpokot'}

df.rename(columns=new_names, inplace=True)

years = [ periodid for periodid in df['periodid']]

In [43]:
#create a data to be supported during prediction(the format is y)
from math import floor
county_monthly_data = []

def county_year_consumption(county):
   
    """This county_year_consumption function gives out the total commodities dispatched a certain complete year
       for a particular county
    """
    #run above function to get the year periods in place
    get_datetime()
    county_monthly_data.clear()
    try:
      for periodid in years:
         val = df.loc[df['periodid']==int(periodid)].sum()
         
         commodity_total = val[county]
         if commodity_total == 0:
            county_monthly_data.append(0)
         else:
            county_monthly_data.append(floor(commodity_total))
         #print(f"{county} has {commodity_total} kits in {periodid}")
      
       
    except Exception as e:
       
       if e.__class__.__name__ == 'KeyError':
          print("Check if the county name is accurate.Or ask the lead developer for solution.")

county_year_consumption('kirinyaga')



In [40]:
import numpy as np
#create a dataframe
#Have the data in a dictionary 
#the first key will be ds while the other y
# Create a dataframe with a datetime column and a corresponding numeric column

#convert the list of datetime to a pandas datetime object
datetime_index  = pd.to_datetime(datetime, format='%Y-%m')
#Extract the end month of the month for each month
end_of_month = datetime_index + pd.offsets.MonthBegin(1)
#print(end_of_month)
dataframe  = {'ds':end_of_month , 'y': county_monthly_data}

new_df = pd.DataFrame(dataframe)

#new_df.head()


In [41]:
#predict using the prophet
from prophet import Prophet
from prophet.diagnostics import performance_metrics, cross_validation

# Instantiate a Prophet object and fit the model to the data
model  = Prophet()

model.fit(new_df)

# Create a dataframe with dates to make a prediction for
future  = model.make_future_dataframe(periods=6, freq='MS')

# Use the model to make a prediction for the next 12 months
forecast  = model.predict(future)

#print the predicted values for the next 12 months
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7))
#model.plot(forecast)

"""#calculate performance metrics using cross-validation
df_cv  = cross_validation(model, initial='730 days', period='180 days', horizon='365 days')

df_p = performance_metrics(df_cv)

#print the performance metrics

print(df_p)"""

12:56:35 - cmdstanpy - INFO - Chain [1] start processing
12:56:36 - cmdstanpy - INFO - Chain [1] done processing


           ds        yhat  yhat_lower  yhat_upper
59 2023-01-01   74.359392  -20.484325  166.057535
60 2023-02-01  102.134585   14.330211  197.219903
61 2023-03-01  116.931968   25.235567  214.340637
62 2023-04-01  173.707020   79.671587  272.499378
63 2023-05-01  137.026558   35.743772  224.878335
64 2023-06-01  126.431857   33.134592  223.053869
65 2023-07-01   71.095873  -17.739074  165.425560


"#calculate performance metrics using cross-validation\ndf_cv  = cross_validation(model, initial='730 days', period='180 days', horizon='365 days')\n\ndf_p = performance_metrics(df_cv)\n\n#print the performance metrics\n\nprint(df_p)"